In [1]:
import pandas as pd
import sqlite3

In [34]:
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()

In [35]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [36]:
for table in tables:
    table_name = table[0]
    print(f"Columns for table {table_name}:")
    df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", conn)
    print(df.columns.tolist())
    print('-' * 50)

Columns for table Player:
['Player_Id', 'Player_Name', 'DOB', 'Batting_hand', 'Bowling_skill', 'Country_Name']
--------------------------------------------------
Columns for table Extra_Runs:
['Match_Id', 'Over_Id', 'Ball_Id', 'Extra_Type_Id', 'Extra_Runs', 'Innings_No']
--------------------------------------------------
Columns for table Batsman_Scored:
['Match_Id', 'Over_Id', 'Ball_Id', 'Runs_Scored', 'Innings_No']
--------------------------------------------------
Columns for table Batting_Style:
['Batting_Id', 'Batting_hand']
--------------------------------------------------
Columns for table Bowling_Style:
['Bowling_Id', 'Bowling_skill']
--------------------------------------------------
Columns for table Country:
['Country_Id', 'Country_Name']
--------------------------------------------------
Columns for table Season:
['Season_Id', 'Man_of_the_Series', 'Orange_Cap', 'Purple_Cap', 'Season_Year']
--------------------------------------------------
Columns for table City:
['City_Id

In [37]:
conn = sqlite3.connect('database.sqlite')
query1 = "SELECT * FROM Player"
df1 = pd.read_sql_query(query1,conn)
print(df1.head())

   Player_Id      Player_Name                  DOB  Batting_hand  \
0          1       SC Ganguly  1972-07-08 00:00:00             1   
1          2      BB McCullum  1981-09-27 00:00:00             2   
2          3       RT Ponting  1974-12-19 00:00:00             2   
3          4        DJ Hussey  1977-07-15 00:00:00             2   
4          5  Mohammad Hafeez  1980-10-17 00:00:00             2   

   Bowling_skill  Country_Name  
0            1.0             1  
1            1.0             4  
2            1.0             5  
3            2.0             5  
4            2.0             6  


In [58]:
query_2 = """
SELECT b.Striker AS batsman, SUM(bs.Runs_Scored) AS total_runs, b.Striker AS Player_Id
FROM Ball_by_Ball b
JOIN Batsman_Scored bs ON b.Match_Id = bs.Match_Id
GROUP BY batsman
ORDER BY total_runs DESC;
"""
df2 = pd.read_sql_query(query_2, conn)
print(df2.head())

   batsman  total_runs  Player_Id
0        8      995223          8
1       21      914446         21
2       57      887853         57
3       40      856072         40
4       46      779183         46


In [60]:
query_3 = """
SELECT
    Striker AS batsman,
    COUNT(CASE WHEN total_runs >= 50 AND total_runs < 100 THEN 1 END) AS fifties,
    COUNT(CASE WHEN total_runs >= 100 THEN 1 END) AS hundreds,
    Striker AS Player_Id
FROM (
    SELECT
        b.Striker,
        b.Match_Id,
        b.Innings_No,
        SUM(bs.Runs_Scored) AS total_runs
    FROM Ball_by_Ball b
    JOIN Batsman_Scored bs ON b.Match_Id = bs.Match_Id AND b.Innings_No = bs.Innings_No
    GROUP BY b.Striker, b.Match_Id, b.Innings_No
) AS batsman_runs
GROUP BY Striker
ORDER BY fifties DESC, hundreds DESC;
"""
df3 = pd.read_sql_query(query_3, conn)
print(df3.head())

   batsman  fifties  hundreds  Player_Id
0      110        3       107        110
1      126        3         8        126
2       21        2       141         21
3      162        2        90        162
4      131        2        17        131


In [61]:
query_4 = """
SELECT
    p.Player_Name AS bowler,
    COUNT(w.Player_Out) AS wickets_taken,
    SUM(bs.Runs_Scored) AS runs_conceded,
    bb.Bowler AS Player_Id
FROM
    Ball_by_Ball bb
JOIN
    Wicket_Taken w ON bb.Match_Id = w.Match_Id AND bb.Over_Id = w.Over_Id AND bb.Ball_Id = w.Ball_Id
JOIN
    Player p ON p.Player_Id = bb.Bowler
LEFT JOIN
    Batsman_Scored bs ON bb.Match_Id = bs.Match_Id AND bb.Innings_No = bs.Innings_No AND bb.Bowler = p.Player_Id
GROUP BY
    bowler
ORDER BY
    wickets_taken DESC, runs_conceded ASC
LIMIT 10;
"""
df4 = pd.read_sql_query(query_4, conn)
print(df4.head())

            bowler  wickets_taken  runs_conceded  Player_Id
0       SL Malinga          34408          41095        194
1         DJ Bravo          27728          33930         71
2    R Vinay Kumar          27652          34401         81
3  Harbhajan Singh          27066          32771         50
4         A Mishra          26150          31436        136


In [62]:
total_batting_metrics = df2.groupby('Player_Id').agg(
    total_runs=('total_runs', 'sum'),
    fifties=('total_runs', lambda x: ((x >= 50) & (x < 100)).sum()),
    hundreds=('total_runs', lambda x: (x >= 100).sum())
).reset_index()

total_bowling_metrics = df4.groupby('Player_Id').agg(
    wickets_taken=('wickets_taken', 'sum'),
    runs_conceded=('runs_conceded', 'sum')
).reset_index()

career_metrics_df = pd.merge(total_batting_metrics, total_bowling_metrics, on='Player_Id', how='outer')
player_names = df1[['Player_Id', 'Player_Name']]
career_metrics_df = pd.merge(career_metrics_df, player_names, on='Player_Id', how='left')

print(career_metrics_df)

     Player_Id  total_runs  fifties  hundreds  wickets_taken  runs_conceded  \
0            1      379585        0         1            NaN            NaN   
1            2      577009        0         1            NaN            NaN   
2            3       35662        0         1            NaN            NaN   
3            4      317211        0         1            NaN            NaN   
4            5       19948        0         1            NaN            NaN   
..         ...         ...      ...       ...            ...            ...   
429        430         262        0         1            NaN            NaN   
430        431       22463        0         1            NaN            NaN   
431        432         332        0         1            NaN            NaN   
432        433        4518        0         1            NaN            NaN   
433        434        1540        0         1            NaN            NaN   

         Player_Name  
0         SC Ganguly  
1    